# The distribution of Einstein Radii of massive galaxy clusters in cosmoDC2

This is a rough estimation of the distribution Einstein Radii of massive galaxy clusters (> 10^14 Msol/h) in cosmoDC2. The calculation is based on analytic NFW (for DM halos) and SIS (for BCGs) models. There are plenty of room to improve the current results, such as involving ellipticities, substructures, los structures, or even particle data from the N-body simulation. Regardless, this is just a starting point for showing how to build mass models of the objects selected from cosmoDC2 and apply the mass models to the scientific applications interesting you. Should you have any questions or suggestions, please do not hesitate to slack me @linan7788626. Hopefully, we can make this notebook useful for your projects.

### Done
- Created mass models of galaxy clusters with SIE and NFW models.
- Calculated the Einstein Radii of the clusters above 10^14 Msol/h (MFOF), with the source plane fixed at zs=3.0.
- Compared the distribution of Re with and without BCGs.
- Predicted how many clusters in LSST data that have Einstein Radii above 30 arcsec. 

### ToDo
- Convert MFOF to M200, or obtain M200 and C200 from addon catalogs for cosmoDC2.
- Add scatters to the parameters of the mass models.
- Other ways to build more realistic mass models according to the information from cosmoDC2.
- Take the redshift distribution of sources into account.
- Compare with available observations if possible.

In [ ]:
%matplotlib inline

import numpy as np
import pylab as pl

In [ ]:
#--------------------------------------------------------------------
# Cosmology model
#

import pyccl as ccl
cosmo = ccl.Cosmology(Omega_c=0.264, Omega_b=0.045, h=0.71, A_s=2.1e-9, n_s=0.96)

vc = 2.998e5            # speed of light, km/s
G = 4.3011790220362e-09 # Gravity constant, Mpc/h (Msun/h)^-1 (km/s)^2
apr = 206269.43         # arcsec per rad

def Dc(z):
    '''
        Comoving distance from redshift 0 to redshift z.
    '''
    res = ccl.comoving_radial_distance(cosmo, 1/(1+z))*cosmo['h']
    return res

def Dc2(z1,z2):
    '''
        Comoving distance from redshift z1 to redshift z2.
    '''
    Dcz1 = ccl.comoving_radial_distance(cosmo, 1/(1+z1))*cosmo['h']
    Dcz2 = ccl.comoving_radial_distance(cosmo, 1/(1+z2))*cosmo['h']
    res = Dcz2-Dcz1+1e-8
    return res
 
def Da(z):
    '''
        Angular diameter distance from redshift 0 to redshift z.
    '''
    res = Dc(z)/(1+z)
    return res
 
def Da2(z1,z2):
    '''
        Angular diameter distance from redshift z1 to redshift z2.
    '''
    res = (Dc(z2)-Dc(z1)+1e-8)/(1.0+z2)
    return res

def Dl(z): 
    '''
        Luminosity distance from redshift 0 to redshift z.
    '''
    res = ccl.luminosity_distance(cosmo, 1/(1+z))*cosmo['h']
    return res 

def SigmaCrit(z1,z2):
    '''
        Critical surface density for the case of lens plane at z1 and source plane at z2.
    '''
    res = (vc*vc/4.0/np.pi/G*Dc(z2)/(Dc(z1)/(1.0+z1))/Dc2(z1,z2))
    return res 

def rho_crit_matter(z):        
    #rho_crit_matter, M_sun Mpc^-3 *h*h
    res = ccl.background.rho_x(cosmo, 1/(1+z), "matter")/cosmo['h']**2
    return res

def dv(z): 
    Omz = ccl.background.omega_x(cosmo, 1/(1+z), "matter")
    ov = 1.0/Omz-1.0
    res = 18.8*np.pi*np.pi*(1.0+0.4093*ov**0.9052)
    return res

def r200_m200(m,z):
#     res = (3.0*m/4.0/np.pi/rho_crit_matter(z)/200.0)**(1.0/3.0)
    res = (3.0*m/4.0/np.pi/rho_crit_matter(z)/dv(z))**(1.0/3.0)
    return res

In [ ]:
def mags_to_vd(mg,mr,zz):
    '''
        Calculate the velocity dispersion of a galaxy according its apparent magnitudes in g and r bands.
        Faber et al. 2007 
        Parker et al. 2007, Table 1
        --- any suggestions on involving more realistic models? ---
    '''
    Dlum     = Dl(zz)
    Mabsr    = mr-5.0*np.log10(Dlum/cosmo.['h'])-25.0
    mrsdss   = Mabsr+0.024*(mg-mr)/0.871
    mrsdss   = mrsdss-0.11
    mrstar   = (-20.44)+(zz-0.1)*1.5
    LbyLstar = 10.0**(-0.4*(mrsdss-mrstar))
    res      = 142.0*LbyLstar**(1./3.)
    return res

def re_sv(sv,z1,z2):
    '''
        Einstein Radius for an galaxy with velocity dispersion sv at redshift z1, 
        and the redshift of source plane is z2. The mass model of the galaxy is 
        Singular Isothermal Ellipsoid (Kormann et al. 1994).
    '''
    res = 4.0*np.pi*(sv**2.0/vc**2.0)*Da2(z1,z2)/Da(z2)*apr
    return res

def sis_kappa(x,y,re,rc=0.0):
    '''
        convergence map of a SIS model,
        inputs are Einstein Radius (re) and the core size (rc).
    '''
    r = np.sqrt(x*x+y*y)
    res = re/(2.0*np.sqrt(r*r+rc*rc))
    return res 

#-----
def c200_m200_HChild2018(m, z):
    '''
        Concentration-Mass relation of dark matter halos, where the inputs are 
        viral mass and the redshift of the halo.
        Child et al. 2018, Table 2.
    '''
    aa = 68.4
    dd = -0.347
    mm = -0.083
    res = aa*(1.0+z)**dd*m**mm
    return res

def nfw_kappa(x1_in,x2_in,c,m,z1,z2):
    
    '''
        convergence map of a NFW halo,
        inputs are: 
            concentration (c), 
            viral mass (m),
            redshift of lens (z1),
            redshift of source (z2)
    '''
    
    r200 = r200_m200(m,z1)
    rs = r200/c
    r = np.sqrt(x1_in*x1_in+x2_in*x2_in)
    xx = r*Da(z1)/apr/rs
    
    rhos = rho_crit_matter(z1)*dv(z1)/3.0*c**3.0/(np.log(1.0+c)-c/(1+c))
    kappas = rs*rhos/SigmaCrit(z1,z2)
    
    x = np.abs(xx)
    x1 = x*x-1.0
    x2 = 2.0/np.sqrt(np.abs(1.0-x*x))
    x3 = np.sqrt(np.abs(1.0-x)/(1+x))
    func_f = x*0.0

    idxa = x>0
    idxb = x<1
    idx1 = idxa&idxb
    func_f[idx1]=1.0/x1[idx1]*(1.0-x2[idx1]*np.arctanh(x3[idx1]))

    idx2 = x==1
    func_f[idx2]=1.0/3.0

    idx3 = x>1.0
    func_f[idx3]=1.0/x1[idx3]*(1.0-x2[idx3]*np.arctan(x3[idx3]))
    
    res = 2.0*kappas*func_f
    return res


In [ ]:
def cart2pol(x, y):
    '''
        convert cartesian coordinates to polar coordinates in a 2D map
    '''
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def find_re(x1_in, x2_in, kappa_in):
    '''
        Find the Einstein Radius of a given convergence map.
        Defination the averaged convergence wthin the Radius equals to 1.0.
        https://arxiv.org/pdf/astro-ph/0304162.pdf
    '''
    rht, pht = cart2pol(x1_in, x2_in)
    rht_r = rht.ravel()
    pht_r = pht.ravel()
    kap_r = kappa_in.ravel()
    idx_pol = np.argsort(((rht_r+1e8)**2.0 + pht_r**2.0))
    kap_sorted = kap_r[idx_pol]
    rht_sorted = rht_r[idx_pol]
    kap_c = np.cumsum(kap_sorted)
    kap_b = kap_c/(np.arange(len(kap_c))+1.0)
    if len(kap_b[np.where(kap_b>1.0)]) < 9:
        # print "There is no Einstein Radii in this lens."
        return 0.0
    else:
        idx_re = np.argmin(np.abs(kap_b-1.0))
        res = rht_sorted[idx_re]
        return res

def make_c_coor(bs,nc):
    '''
        Draw the mesh grids for a bs*bs box with nc*nc pixels
    '''
    ds = bs/nc
    xx01 = np.linspace(-bs/2.0,bs/2.0-ds,nc)+0.5*ds
    xx02 = np.linspace(-bs/2.0,bs/2.0-ds,nc)+0.5*ds
    xi2,xi1 = np.meshgrid(xx01,xx02)
    return xi1,xi2

In [ ]:
'''
    Load in cosmoDC2
'''

import GCRCatalogs
# areav100=805.5121
# gc = GCRCatalogs.load_catalog('cosmoDC2_v1.0')

areav114=487.6296 
gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

In [ ]:
%%time
'''
    Grab the parameters for the calculation of Einstein Radii,
    Selection functions is central galaxies with halo mass larger than 1e14 Msol/h.
'''
gals_data_dict = gc.get_quantities(['galaxyID', 
                                    'mag_true_g_lsst', 
                                    'mag_true_r_lsst', 
                                    'size_true',
                                    'size_minor_true',
                                    'position_angle_true', 
                                    'redshift_true', 
                                    'stellar_mass', 
                                    'halo_mass', 
                                    'is_central'],
                                    filters=['halo_mass>=1e14', 
                                             'is_central==True'])

In [ ]:
'''
    print out how many halos with mass above 1e14 Msol/h
'''

print("There are", len(gals_data_dict['galaxyID']), "galaxy clusters in cosmoDC2 v1.1.4 above $10^{14} M_{\odot}/h$")

In [ ]:
'''
    convert tables to arrays, and set the redshift of sources at z = 3.0
'''

z_ref = 3.0
idx_sort = np.argsort(gals_data_dict['halo_mass'])[::-1]

m200_main = gals_data_dict['halo_mass'][idx_sort]
magg_main = gals_data_dict['mag_true_g_lsst'][idx_sort]
magr_main = gals_data_dict['mag_true_r_lsst'][idx_sort]
zl_main   = gals_data_dict['redshift_true'][idx_sort]

vd_main   = mags_to_vd(magg_main, magr_main, zl_main)
re_main   = re_sv(vd_main, zl_main, z_ref)
r200_main = r200_m200(m200_main, zl_main)
c200_main = c200_m200_HChild2018(m200_main, zl_main)

In [ ]:
'''
Calculate the Einstein Radii, the runtime is about one hour.
'''

nnn = 1024
re_sie = []
re_nfw = []
re_tot = []

for i in range(len(vd_main)):
    # box size is set to be 0.5 viral radius (in the unites of arcsec)
    bsx = r200_main[i]*0.5/Da(zl_main[i])*apr
    xi1, xi2 = make_c_coor(bsx,nnn)

    # calculate the Einstein radius of the BCG only
    kappa_bcgs = sis_kappa(xi1,xi2,re_main[i],0.0)
    re_sie_tmp = find_re(xi1, xi2, kappa_bcgs)
    re_sie.append(re_sie_tmp)
    
    # calculate the Einstein radius of the Dark matter halo only
    kappa_halo = nfw_kappa(xi1,xi2,c200_main[i],m200_main[i],zl_main[i],z_ref)
    re_nfw_tmp = find_re(xi1, xi2, kappa_halo)
    re_nfw.append(re_nfw_tmp)
    
    # calculate the Einstein Radius of the cluster with both BCG and DM halo
    kappa_tot = kappa_bcgs + kappa_halo
    re_tot_tmp = find_re(xi1, xi2, kappa_tot)
    re_tot.append(re_tot_tmp)
    
re_tot_arr = np.array(re_tot)
re_nfw_arr = np.array(re_nfw)
re_sie_arr = np.array(re_sie)

In [ ]:
'''
    scale up to the survey area of LSST, 
    and print out how many clusters with Einstein Radii larger than 30 arcsec 
'''

arealsst = 18000 # degree^2

print("In LSST data, there are roughly", 
      int(len(re_tot_arr[np.where(re_tot_arr>30)])*arealsst/areav114), 
      "galaxy cluster having Einstein Radii larger than 30 arcsec.")

In [ ]:
'''
The Distribution of Einstein Radii in Linear Space
'''

import seaborn as sns;sns.set()
pl.figure(figsize=(10, 7))
sns.distplot(re_tot_arr[np.where(re_tot_arr>0.01)],
             kde_kws={"color": "b", 
                      "lw": 3, 
                      "label": "DM+BCG"},
             hist_kws={"histtype": "bar", 
                       "linewidth": 0, 
                       "alpha": 0.5, 
                       "color": "b"})

sns.distplot(re_nfw_arr[np.where(re_nfw_arr>0.01)],
             kde_kws={"color": "r", 
                      "lw": 3, 
                      "label": "DM Only"},
             hist_kws={"histtype": "bar", 
                       "linewidth": 0, 
                       "alpha": 0.5, 
                       "color": "r"})

pl.xlim(0.5,35);

In [ ]:
'''
The distribution of Einstein Radii in logarithmic Space. 
'''

bins = 10**(np.linspace(0,np.log10(35.), 20))

import seaborn as sns;sns.set()
pl.figure(figsize=(10, 7))
sns.distplot(re_tot_arr[np.where(re_tot_arr>0.5)], bins=bins, 
             kde_kws={"color": "b", 
                      "lw": 3, 
                      "label": "DM+BCG"},
             hist_kws={"histtype": "bar", 
                       "linewidth": 0, 
                       "alpha": 0.5, 
                       "color": "b"})

sns.distplot(re_nfw_arr[np.where(re_nfw_arr>0.5)], bins=bins, 
             kde_kws={"color": "r", 
                      "lw": 3, 
                      "label": "DM Only"},
             hist_kws={"histtype": "bar", 
                       "linewidth": 0, 
                       "alpha": 0.5,
                       "color": "r"})

pl.xscale('log')
pl.xlim(1.0,35);